In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow as tf
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

K.image_dim_ordering = 'th'

seed = 7
np.random.seed(seed)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
num_classes

10

In [5]:
# Create the model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape = (32, 32, 3), padding= 'same', activation = 'relu', kernel_constraint = maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation = 'relu', padding = 'same', kernel_constraint = maxnorm(3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(512, activation = 'relu', kernel_constraint = maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

In [6]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 11, 11, 32)        896       
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 512)               66048     
                                                        

In [7]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, verbose=2)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 - 7s - loss: 1.9501 - acc: 0.2876 - val_loss: 1.6955 - val_acc: 0.3969 - 7s/epoch - 135us/sample
Epoch 2/25
50000/50000 - 6s - loss: 1.7017 - acc: 0.3851 - val_loss: 1.5578 - val_acc: 0.4379 - 6s/epoch - 120us/sample
Epoch 3/25
50000/50000 - 6s - loss: 1.6009 - acc: 0.4194 - val_loss: 1.4982 - val_acc: 0.4582 - 6s/epoch - 124us/sample
Epoch 4/25
50000/50000 - 5s - loss: 1.5403 - acc: 0.4420 - val_loss: 1.4766 - val_acc: 0.4667 - 5s/epoch - 109us/sample
Epoch 5/25
50000/50000 - 6s - loss: 1.5019 - acc: 0.4553 - val_loss: 1.4588 - val_acc: 0.4680 - 6s/epoch - 116us/sample
Epoch 6/25
50000/50000 - 6s - loss: 1.4742 - acc: 0.4662 - val_loss: 1.3979 - val_acc: 0.4888 - 6s/epoch - 116us/sample
Epoch 7/25
50000/50000 - 6s - loss: 1.4552 - acc: 0.4704 - val_loss: 1.3889 - val_acc: 0.4936 - 6s/epoch - 114us/sample
Epoch 8/25
50000/50000 - 6s - loss: 1.4313 - acc: 0.4813 - val_loss: 1.3647 - val_acc: 0.5049 - 6s/epoch - 111us/sample
Epoch 9/25
50000/50000 - 6s - loss: 1.4174 - acc: 0

In [8]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 55.16%
